This competition is very interesting and special because neural networks perform better than boosting models on tabular data.

This can be explained in two ways: 
- The structure of the data may not be adapted to boosting algorithms (it seems that the order is not realy important between the cells features meaning that you can permute c-x,c-y for some sig_ids without changing results).

- and also the fact that the boosting algorithms used so far on pubic kernels use only one target and not all 206 at a time.

In this notebook I directly implement a lgbm on the 206 labels (it's run fast).

To achieve this I need to use some tricks that allow the lgbm to learn simultaneously:

- The lgbm doesn't accept a matrix (n_sample,206) as an input target.
- You have to give it a vector (n_sample,1) in the input and recover (n_sample,206) in your loss function.

let's dive in it !****

<h1><font color="red"> Please upvote if you find this helpfull !  </font></h1>

In [ ]:
import os
import gc
import sys
import time
import random
import operator
import typing as tp
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd,numpy as np

from tqdm import *
from datetime import date, datetime 

import lightgbm as lgb


from sklearn.metrics import *

import warnings
warnings.simplefilter("ignore")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [ ]:
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
Train_target = pd.read_csv("../input/lish-moa/train_targets_scored.csv")
Train = pd.read_csv("../input/lish-moa/train_features.csv")
Test = pd.read_csv("../input/lish-moa/test_features.csv")
SampleSubmission = pd.read_csv("../input/lish-moa/sample_submission.csv")

Train.shape,Test.shape,SampleSubmission.shape,Train_target.shape

In [ ]:
TARGETS = Train_target.columns[1:]

In [ ]:
Train = Train.merge(Train_target,how='left',on="sig_id")
Test = Test.merge(SampleSubmission,how='left',on="sig_id")
Test.loc[:,SampleSubmission.columns[1:]] = 0

In [ ]:
Train.head(2)

In [ ]:
def Metric(y_true, y_pred):
    y_pred_clip = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = - np.mean(np.mean(y_true * np.log(y_pred_clip) + (1 - y_true) * np.log(1 - y_pred_clip), axis = 1))
    return np.round(loss,5)

def FE(data,scale=True):
    
    # Categories Transform
    data["sig_type"] = data['cp_type'].astype(str) + '_' + data['cp_dose'].astype(str) + '_' + data['cp_time'].astype(str)
    data.loc[:, 'cp_type'] = data.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    data.loc[:, 'cp_dose'] = data.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})

    data["Label"] = data[TARGETS].apply(lambda x : GetLabel(x.values),axis=1)
    
    return data

def GetLabel(x):
    arg = np.argwhere(x==1)
    return " ".join(np.asarray(TARGETS[arg]).reshape(-1))

In [ ]:
Train['Train'] = 1
Test['Train'] = 0
Data = pd.concat([Train,Test],axis=0)

del Train,Test
gc.collect()

In [ ]:
Data = FE(Data)

In [ ]:
Train = Data[(Data.Train==1) ].reset_index(drop=True)
Test = Data[Data.Train==0].reset_index(drop=True)
Train.shape,Test.shape

# Targets Dictionnary

**(n_sample,1) ---------------------> (n_sample,206)**

In [ ]:
LABELS_ ={x:y for x,y in  zip(Train['Label'].unique(),np.arange(len(Train['Label'].unique())))}
Train['Target'] = Train["Label"].map(LABELS_)

In [ ]:
Unique = Train[TARGETS.tolist()+["Target"]].drop_duplicates("Target")
Unique.head()

In [ ]:
LABELS ={x:Unique.loc[Unique.Target==x,TARGETS].values.reshape(206) for x in Train['Target'].unique()}

In [ ]:
LABELS[0]

### Custom Loss

* **The log loss function is sum of :**  $y*ln(p)+(1−y)*ln(1−p)$

> **$$logloss = \sum _{i=1}^{n=N}y_i*ln(p_i)+(1−y_i)*ln(1−p_i)$$**

> where $$p=\frac{1}{1+e−\beta x}$$

* **The gradient (with respect to $p$) is :**

$$\frac{∂logloss}{∂p}=\frac{p−y}{(p−1)p}$$

**and the derivative of sigmoid is:**

$$\frac{∂p}{∂x}=p(1−p)$$

**So :**

$$\frac{∂logloss}{∂x}=\frac{∂logloss}{∂p}*\frac{∂p}{∂x}=p−y$$

* **The hessian (with respect to $p$) is:**

$$\frac{∂^2 logloss}{∂ x^2}=\frac{∂}{∂x}(\frac{∂logloss}{∂x})=\frac{∂}{∂x}(p−y)=\frac{∂p}{∂x}=p(1−p)$$

* **With Penalization** $\beta$

$$\frac{∂logloss}{∂x}=p*((\beta -1)*y+1)−\beta*y$$

$$\frac{∂^2 logloss}{∂ x^2}=((\beta -1)*y+1)*p(1−p)$$

In [ ]:
def sigmoid(x):
    X = np.where(x >= 0,
                     1. / (1. + np.exp(-x)),
                     np.exp(x) / (1. + np.exp(x)))
    return X

class MoaLGBM:
    """
    Custom Loss for LightGBM.
    
    * Objective: return grad & hess
    * Evaluation: return log_loss
    """
    
    def __init__(self, epsilon: float=1,beta=2) -> None:
        """Initialize."""
        self.name = "moa_loss"
        self.n_class = 206  
        self.epsilon = epsilon
        self.beta = beta
    
    def __call__(self, preds: np.ndarray, labels: np.ndarray, weight: tp.Optional[np.ndarray]=None) -> float:
        """Calc loss."""
        preds = sigmoid(preds)
        loss =  Metric(labels, preds)
        return loss
    
    def _calc_grad_and_hess(
        self, preds: np.ndarray, labels: np.ndarray, weight: tp.Optional[np.ndarray]=None
    ) -> tp.Tuple[np.ndarray]:
        """Calc Grad and Hess"""
        
        preds = sigmoid(preds)
        grad = np.zeros_like(preds)
        hess = np.zeros_like(preds)
        
        grad = preds * ((self.beta -1) * labels + 1) - self.beta * labels
        hess = ((self.beta - 1) * labels + 1) * preds * (1.0 - preds)

        if weight is not None:
            grad = grad * weight[:, None]
            hess = hess * weight[:, None]
        return grad, hess
    
    def return_loss(self, preds: np.ndarray, data: lgb.Dataset) -> tp.Tuple[str, float, bool]:
        """Return Loss for lightgbm"""
        labels = data.get_label()
        weight = data.get_weight()
        n_example = len(labels)
        
        # Replace input with true vector
        labels_ = np.asarray([LABELS[i] for i in labels])

        preds = preds.reshape(self.n_class, n_example).T
        # calc loss
        loss = self(preds, labels_, weight)
        
        return self.name, loss, False
    
    def return_grad_and_hess(self, preds: np.ndarray, data: lgb.Dataset) -> tp.Tuple[np.ndarray]:
        """Return Grad and Hess for lightgbm"""
        labels = data.get_label()
        weight = data.get_weight()
        n_example = len(labels)
        # Replace input with true vector
        labels_ = np.asarray([LABELS[i] for i in labels])

        preds = preds.reshape(self.n_class, n_example).T
        # # calc grad and hess.
        grad, hess =  self._calc_grad_and_hess(preds, labels_, weight)

        # # reshape grad, hess: (n_example, n_class) => (n_class, n_example) => (n_class * n_example,) 
        grad = grad.T.reshape(n_example * self.n_class)
        hess = hess.T.reshape(n_example * self.n_class)
        
        return grad, hess

<h1><font color="red">  you can increase lambda_l1 to avoid overfiting</font></h1>

In [ ]:
lgb_model_param = {
    'num_class': 206,
    'metric': 'None',
    'boost_from_average':'true',
    'boosting_type': 'gbdt',
    
#     "lambda_l1":0.5,
#     "lambda_l2":0.5,
#     "learning_rate":0.1,
    'seed': 8,
    'max_depth': -1,
    'tree_learner': 'serial',
    'verbosity': -1,
}
lgb_fit_param = {
    "num_boost_round": 500,
    "verbose_eval":1,
    "early_stopping_rounds": 10,
}

# Penalization Beta
beta = 2.5
LgbmLoss = MoaLGBM(beta)

In [ ]:
Top_Feat = ["cp_type",'cp_time', 'cp_dose', 'g-0', 'g-1', 'g-2', 'g-3', 'g-4', 'g-6', 'g-8', 'g-11', 'g-12', 'g-13', 'g-14', 'g-15', 'g-16',
 'g-17', 'g-18', 'g-19', 'g-20', 'g-21', 'g-22', 'g-23', 'g-24', 'g-26', 'g-27', 'g-28', 'g-29', 'g-30', 'g-32', 'g-33', 'g-34', 'g-35',
  'g-36', 'g-37', 'g-38', 'g-39', 'g-40', 'g-41', 'g-43', 'g-44', 'g-45', 'g-46', 'g-47', 'g-48', 'g-49', 'g-50', 'g-51', 'g-52', 'g-53', 
  'g-55', 'g-56', 'g-57', 'g-58', 'g-59', 'g-60', 'g-61', 'g-62', 'g-63', 'g-64', 'g-65', 'g-66', 'g-67', 'g-68', 'g-69', 'g-70', 'g-71',
   'g-72', 'g-73', 'g-75', 'g-76', 'g-77', 'g-78', 'g-79', 'g-80', 'g-81', 'g-83', 'g-84', 'g-85', 'g-86', 'g-87', 'g-88', 'g-89', 'g-90', 
   'g-91', 'g-92', 'g-93', 'g-94', 'g-95', 'g-96', 'g-97', 'g-98', 'g-99', 'g-100', 'g-101', 'g-102', 'g-103', 'g-104', 'g-105', 'g-106',
    'g-107', 'g-108', 'g-109', 'g-110', 'g-111', 'g-112', 'g-113', 'g-114', 'g-115', 'g-117', 'g-118', 'g-119', 'g-120', 'g-121', 'g-122', 
    'g-123', 'g-124', 'g-125', 'g-126', 'g-127', 'g-128', 'g-129', 'g-130', 'g-133', 'g-134', 'g-135', 'g-136', 'g-137', 'g-138', 'g-139', 
    'g-140', 'g-141', 'g-142', 'g-143', 'g-144', 'g-146', 'g-147', 'g-148', 'g-149', 'g-150', 'g-151', 'g-152', 'g-153', 'g-154', 'g-155',
     'g-156', 'g-157', 'g-158', 'g-159', 'g-160', 'g-161', 'g-162', 'g-163', 'g-164', 'g-165', 'g-166', 'g-167', 'g-168', 'g-169', 'g-170', 
     'g-171', 'g-172', 'g-173', 'g-174', 'g-175', 'g-176', 'g-177', 'g-178', 'g-179', 'g-180', 'g-181', 'g-182', 'g-183', 'g-184', 'g-185', 
     'g-186', 'g-187', 'g-188', 'g-189', 'g-190', 'g-191', 'g-192', 'g-194', 'g-195', 'g-196', 'g-197', 'g-199', 'g-200', 'g-201', 'g-202', 
     'g-203', 'g-205', 'g-206', 'g-207', 'g-208', 'g-209', 'g-210', 'g-211', 'g-212', 'g-213', 'g-214', 'g-215', 'g-216', 'g-217', 'g-218', 
     'g-220', 'g-221', 'g-222', 'g-223', 'g-224', 'g-225', 'g-226', 'g-227', 'g-228', 'g-229', 'g-230', 'g-231', 'g-232', 'g-233', 'g-234', 
     'g-235', 'g-236', 'g-237', 'g-239', 'g-240', 'g-241', 'g-242', 'g-243', 'g-244', 'g-245', 'g-246', 'g-247', 'g-248', 'g-249', 'g-250',
      'g-251', 'g-252', 'g-253', 'g-254', 'g-255', 'g-256', 'g-257', 'g-258', 'g-259', 'g-260', 'g-261', 'g-262', 'g-263', 'g-264', 'g-265',
       'g-266', 'g-268', 'g-269', 'g-270', 'g-271', 'g-272', 'g-273', 'g-274', 'g-275', 'g-276', 'g-277', 'g-278', 'g-279', 'g-280', 'g-281', 
       'g-282', 'g-283', 'g-284', 'g-285', 'g-286', 'g-287', 'g-288', 'g-289', 'g-291', 'g-292', 'g-293', 'g-295', 'g-297', 'g-298', 'g-299',
        'g-300', 'g-301', 'g-302', 'g-303', 'g-304', 'g-305', 'g-306', 'g-307', 'g-308', 'g-309', 'g-311', 'g-312', 'g-313', 'g-314', 'g-315', 
        'g-316', 'g-317', 'g-318', 'g-319', 'g-320', 'g-321', 'g-322', 'g-323', 'g-324', 'g-325', 'g-326', 'g-327', 'g-328', 'g-329', 'g-330',
         'g-331', 'g-332', 'g-333', 'g-334', 'g-335', 'g-336', 'g-337', 'g-338', 'g-339', 'g-340', 'g-341', 'g-342', 'g-343', 'g-344', 'g-345', 
         'g-346', 'g-347', 'g-348', 'g-349', 'g-350', 'g-351', 'g-352', 'g-353', 'g-354', 'g-355', 'g-356', 'g-357', 'g-358', 'g-359', 'g-360', 
         'g-361', 'g-362', 'g-363', 'g-364', 'g-365', 'g-366', 'g-367', 'g-368', 'g-371', 'g-372', 'g-373', 'g-374', 'g-375', 'g-376', 'g-377', 
         'g-378', 'g-379', 'g-380', 'g-381', 'g-382', 'g-383', 'g-384', 'g-385', 'g-387', 'g-388', 'g-389', 'g-390', 'g-391', 'g-392', 'g-393', 
         'g-394', 'g-395', 'g-396', 'g-397', 'g-398', 'g-399', 'g-400', 'g-401', 'g-402', 'g-403', 'g-404', 'g-405', 'g-406', 'g-408', 'g-409', 'g-410', 
         'g-411', 'g-412', 'g-413', 'g-414', 'g-415', 'g-416', 'g-417', 'g-418', 'g-419', 'g-420', 'g-421', 'g-422', 'g-423', 'g-424', 'g-425', 'g-426',
          'g-427', 'g-428', 'g-429', 'g-431', 'g-432', 'g-433', 'g-434', 'g-435', 'g-436', 'g-437', 'g-439', 'g-440', 'g-441', 'g-442', 'g-443', 'g-444',
           'g-445', 'g-446', 'g-447', 'g-450', 'g-451', 'g-453', 'g-454', 'g-455', 'g-456', 'g-457', 'g-458', 'g-459', 'g-460', 'g-461', 'g-462', 'g-463',
            'g-464', 'g-465', 'g-466', 'g-467', 'g-468', 'g-469', 'g-470', 'g-471', 'g-472', 'g-473', 'g-474', 'g-475', 'g-476', 'g-478', 'g-479',
             'g-480', 'g-481', 'g-482', 'g-483', 'g-484', 'g-485', 'g-486', 'g-487', 'g-488', 'g-489', 'g-490', 'g-491', 'g-492', 'g-493', 'g-495',
              'g-497', 'g-498', 'g-499', 'g-500', 'g-502', 'g-503', 'g-504', 'g-506', 'g-507', 'g-508', 'g-509', 'g-510', 'g-511', 'g-512', 'g-515',
               'g-516', 'g-517', 'g-518', 'g-519', 'g-520', 'g-521', 'g-522', 'g-523', 'g-524', 'g-525', 'g-527', 'g-528', 'g-529', 'g-531', 'g-532',
                'g-533', 'g-535', 'g-536', 'g-537', 'g-538', 'g-539', 'g-540', 'g-541', 'g-542', 'g-543', 'g-544', 'g-546', 'g-547', 'g-548', 'g-549', 
                'g-551', 'g-554', 'g-556', 'g-557', 'g-558', 'g-559', 'g-562', 'g-563', 'g-564', 'g-565', 'g-566', 'g-567', 'g-568', 'g-569', 'g-570', 
                'g-571', 'g-572', 'g-574', 'g-575', 'g-577', 'g-578', 'g-579', 'g-580', 'g-581', 'g-582', 'g-583', 'g-584', 'g-585', 'g-586', 'g-587',
                 'g-588', 'g-589', 'g-590', 'g-591', 'g-592', 'g-593', 'g-594', 'g-596', 'g-597', 'g-598', 'g-599', 'g-603', 'g-604', 'g-605', 'g-606',
                  'g-608', 'g-609', 'g-610', 'g-612', 'g-613', 'g-614', 'g-615', 'g-616', 'g-617', 'g-618', 'g-619', 'g-620', 'g-621', 'g-622', 'g-623', 
                  'g-624', 'g-625', 'g-626', 'g-627', 'g-628', 'g-629', 'g-630', 'g-631', 'g-632', 'g-633', 'g-634', 'g-635', 'g-636', 'g-638', 'g-639',
                   'g-640', 'g-641', 'g-642', 'g-643', 'g-644', 'g-645', 'g-646', 'g-647', 'g-648', 'g-649', 'g-651', 'g-652', 'g-653', 'g-655', 'g-656',
                    'g-657', 'g-658', 'g-659', 'g-660', 'g-661', 'g-662', 'g-663', 'g-664', 'g-665', 'g-666', 'g-667', 'g-668', 'g-669', 'g-670', 'g-671',
     'g-672', 'g-673', 'g-674', 'g-675', 'g-676', 'g-677', 'g-678', 'g-679', 'g-680', 'g-681', 'g-682', 'g-683', 'g-684', 'g-685', 'g-686', 'g-688', 'g-689',
     'g-690', 'g-691', 'g-692', 'g-693', 'g-694', 'g-696', 'g-697', 'g-698', 'g-699', 'g-701', 'g-702', 'g-704', 'g-705', 'g-706', 'g-707', 'g-708', 'g-710',
     'g-711', 'g-713', 'g-714', 'g-715', 'g-717', 'g-718', 'g-720', 'g-721', 'g-722', 'g-723', 'g-724', 'g-725', 'g-726', 'g-727', 'g-728', 'g-729', 'g-730',
     'g-731', 'g-732', 'g-734', 'g-735', 'g-736', 'g-737', 'g-739', 'g-740', 'g-741', 'g-742', 'g-743', 'g-744', 'g-745', 'g-746', 'g-747', 'g-748', 'g-749', 
     'g-750', 'g-751', 'g-752', 'g-753', 'g-754', 'g-756', 'g-757', 'g-758', 'g-759', 'g-760', 'g-761', 'g-762', 'g-763', 'g-764', 'g-765', 'g-766', 'g-767', 
     'g-768', 'g-769', 'g-770', 'g-771', 'c-0', 'c-1', 'c-2', 'c-4', 'c-5', 'c-6', 'c-7', 'c-8', 'c-9', 'c-10', 'c-11', 'c-12', 'c-13', 'c-14', 'c-15', 'c-17',
     'c-18', 'c-19', 'c-20', 'c-21', 'c-22', 'c-23', 'c-25', 'c-26', 'c-27', 'c-28', 'c-29', 'c-30', 'c-31', 'c-33', 'c-34', 'c-36', 'c-38', 'c-39', 'c-40', 
     'c-41', 'c-42', 'c-43', 'c-44', 'c-46', 'c-47', 'c-48', 'c-50', 'c-51', 'c-52', 'c-53', 'c-54', 'c-55', 'c-56', 'c-57', 'c-59', 'c-60', 'c-62', 'c-63', 
     'c-64', 'c-65', 'c-66', 'c-67', 'c-70', 'c-71', 'c-72', 'c-73', 'c-75', 'c-76', 'c-77', 'c-79', 'c-80', 'c-81', 'c-83', 'c-84', 'c-85', 'c-86', 'c-87', 
     'c-89', 'c-91', 'c-92', 'c-93', 'c-94', 'c-95', 'c-96', 'c-97', 'c-98', 'c-99']

In [ ]:
K = 7
preds = []
skf = MultilabelStratifiedKFold(n_splits=K)
splits = list(skf.split(X=Train, y=Train[TARGETS]))
for i, (train_idx, val_idx) in enumerate(splits):
    print(f"\n-------------   Fold {i + 1} / {K}  -------------\n")

    trn_data=lgb.Dataset(Train.loc[train_idx,Top_Feat], label=Train.loc[train_idx,"Target"])
    val_data=lgb.Dataset(Train.loc[val_idx,Top_Feat], label=Train.loc[val_idx,"Target"])
    clf=lgb.train(lgb_model_param, trn_data, valid_sets = [trn_data, val_data],
                  fobj=LgbmLoss.return_grad_and_hess,
                  feval=LgbmLoss.return_loss,
                  learning_rates=lambda iter: 0.1 * (0.99 ** iter),
                  verbose_eval=1, early_stopping_rounds = 10)
    
    preds.append(sigmoid(clf.predict(Test[Top_Feat],num_iteration=clf.best_iteration)))
    

In [ ]:
Test.loc[:,TARGETS] = np.mean(preds,0)

In [ ]:
Test.loc[Test.cp_type=="ctl_vehicle",TARGETS] = 0

In [ ]:
Test[["sig_id"]+TARGETS.tolist()].to_csv("submission.csv", index=False)